In [169]:
pip install -U scikit-learn

In [170]:
import json

In [171]:
file_name = "/content/Books_small_10000.json"

In [172]:
import random
class Sentiment:
  negative = "NEGATIVE"
  neutral = "NEUTRAL"
  positive = "POSITIVE"

class Review:
  def __init__(self, text, star):
      self.text = text  # the written review
      self.star = star # the given star
      self.sentiment = self.get_sentiment()
  
  def get_sentiment(self):
    if self.star <= 2:
      return Sentiment.negative
    elif self.star == 3 :
      return Sentiment.neutral
    else: 
      return Sentiment.positive

class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.negative, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.positive, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

In [173]:
reviews = []

with open (file_name) as f: 
  for line in f:
    #print(line)
    review = json.loads(line) # as its a json data , so you have to load it with json

    #print(review['reviewText']) # and we want the reviewText so you can find that in this manner. 
    #print(review['overall'])

    reviews.append(Review(review['reviewText'], review['overall']))  # this line will add all the reviews and star in your file

In [174]:
reviews[6].star # you get the particular star

2.0

In [175]:
reviews[6].text #you get the particular review

'The book has the fevered intensity of Oliver Stone\'s movie "JFK", as the author tracks his own descent into the tangle of conspiracy theories and suspects that surround the DB Cooper hijacking case.  The book itself bounces back and forth in time, alternating between the hijacking itself, the author\'s journey, and the lives of the various suspects.  It becomes difficult to keep track of the characters in the story, and the book ultimately spirals away into paranoia.  An unsatisfying read that ends in a shoulder shrug that obscures more than it reveals.'

In [176]:
len(reviews) # total number of reviews. 

10000

Prep data

In [177]:
from sklearn.model_selection import train_test_split 

somnetimes  we train the model with entire data set and thats not a good staytegy , so you split the data set in two part where part of the data set is used for training and part of them  is used to test your model. 

SO HERE HOW IT WORKS

you need to train your machuine learning model. so you train you model with 80% of your data and the other 20% is for testing the model

In [178]:
training, test = train_test_split(reviews, test_size= 0.33, random_state = 42)

train_container = ReviewContainer(training)

test_container = ReviewContainer(test)

In [179]:

train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.positive))
print(train_y.count(Sentiment.negative))


436
436


**Bag of word vectorization**

We call vectorization the general process of turning a collection of text documents into numerical feature vectors. This specific strategy (tokenization, counting and normalization) is called the Bag of Words or “Bag of n-grams” representation.
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [180]:
# example of sklearn bags of vectorization, this cell is not a part of the code
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
     'This is the first document.',
     'This document is the second document.',
    'And this is the third one.',
     'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print("unique words : ")
print(vectorizer.get_feature_names_out(),'\n')

print("tokenization of words in matrix : ")
print(X.toarray(),'\n')

# to select more than 1 word
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(corpus)
print("unique words of two : ")
print(vectorizer2.get_feature_names_out(),'\n')

print("tokenization of words in matrix : ")
print(X2.toarray())

unique words : 
['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this'] 

tokenization of words in matrix : 
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]] 

unique words of two : 
['and this' 'document is' 'first document' 'is the' 'is this'
 'second document' 'the first' 'the second' 'the third' 'third one'
 'this document' 'this is' 'this the'] 

tokenization of words in matrix : 
[[0 0 1 1 0 0 1 0 0 0 0 1 0]
 [0 1 0 1 0 1 0 1 0 0 1 0 0]
 [1 0 0 1 0 0 0 0 1 1 0 1 0]
 [0 0 1 0 1 0 1 0 0 0 0 0 1]]


In [181]:
#starts from here
from sklearn.feature_extraction.text import CountVectorizer

In [182]:
# transforming the the word in bag of vectorization
vectorizer = CountVectorizer()
vectorizer.fit_transform(train_x) # it make all the text in the token in a token format 0/1

test_x_vectors = vectorizer.transform(test_x)

In [183]:
print(train_x[0])

A well-written chronically of what shaped Senator Warren into a person who never gives up a righteous fight.  It is never boring but truly inspiring.  I've tried to do some worthwhile things in my life, but in comparison to Elizabeth Warren I'm a slacker.  Thank God for Senator Warren and her very large band of assistants and backers.  I'm ready to read her other books.


now see it in the token form where it transform into a matrix.

In [184]:
train_x_vectors = vectorizer.fit_transform(train_x)
print(train_x_vectors[0].toarray())

[[0 0 0 ... 0 0 0]]


**Classification**

**Linear SVM**

Predicting using linear svm classsification

In [185]:
from sklearn import svm

In [186]:
clf_svm = svm.SVC(kernel = 'linear')
clf_svm.fit(train_x_vectors, train_y)

print(test_x[0])
print(clf_svm.predict(test_x_vectors[0]))
print()

print(test_x[7])
print(clf_svm.predict(test_x_vectors[7]))
print()

print(test_x[8])
print(clf_svm.predict(test_x_vectors[8]))
print()


First of all, this book needs some serious editing.  This book probably should not have been sold in it's  current version.  The story itself is a little unbelievable.  Billionaire brothers who beat up their fathers don't exist in most people's lives.  The storyline or what I could make out to be the storyline is really weak.  Save your money.
['NEGATIVE']

Not really impressed. Finished reading it because I purchased it. The misspelled words and weird phrases were very distracting to me.
['NEGATIVE']

I really have enjoyed this series and can't wait for the next one to come out. I especially liked that the author gave Kid and Jason their own stories, even though they occurred at the same time. She could have easily put them both into one book and jumped between locations. I am so glad she didn't.The last two installments have not had as much sensuality as the previous 3 books, but the story line is there, so you don't necessarily miss it. I do like how she has not just dropped the oth

**Decision Tree**

Predicting using decision tree classifier

In [187]:
from sklearn.tree import DecisionTreeClassifier

In [188]:
clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)


for i in range (5,10):
  print(clf_dec.predict(test_x_vectors[i]))

['POSITIVE']
['NEGATIVE']
['NEGATIVE']
['NEGATIVE']
['POSITIVE']


**Naive Bayes**

Using naive bayes classificaton

In [189]:
from sklearn.naive_bayes import GaussianNB

In [190]:
#clf_gnb = GaussianNB()
#clf_gnb.fit(train_x_vectors, train_y)
#for i in range (5,10):
  #print(clf_gnb.predict(test_x_vectors[i]))

**Logistic Regression**

Using logistic regression

In [191]:
from sklearn.linear_model import LogisticRegression

In [192]:
clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)


LogisticRegression()

In [193]:
for i in range (5,10):
  print(clf_log.predict(test_x_vectors[i]))

['NEGATIVE']
['NEGATIVE']
['NEGATIVE']
['POSITIVE']
['NEGATIVE']


**Evaluation**

See which model is  doing better / providing more accuracy

In [194]:
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
#print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))


0.7980769230769231
0.6418269230769231
0.8149038461538461


**F1 SCORE**

F1-score is one of the most important evaluation metrics in machine learning. It elegantly sums up the predictive performance of a model by combining two otherwise competing metrics — precision and recall


In [195]:
from sklearn.metrics import f1_score

F1 score for svm 

In [196]:
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels = [Sentiment.positive,Sentiment.neutral, Sentiment.negative])


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


array([0.8028169 , 0.        , 0.79310345])

F1 score for decision tree

In [197]:
f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels = [Sentiment.positive,Sentiment.neutral, Sentiment.negative])


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


array([0.64439141, 0.        , 0.63922518])

F1 score for logistic regression

In [198]:
f1_score(test_y, clf_log.predict(test_x_vectors), average=None, labels = [Sentiment.positive,Sentiment.neutral, Sentiment.negative])


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


array([0.82051282, 0.        , 0.808933  ])

In [199]:
train_y[0 : 10]

['POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE',
 'NEGATIVE',
 'POSITIVE',
 'POSITIVE',
 'POSITIVE']

In [201]:
test_set = ['bad very good', " great", 'horrible waste of time']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)

array(['NEGATIVE', 'POSITIVE', 'NEGATIVE'], dtype='<U8')